# Download sentinel data from Google Earth Engine 
This script permits to download satellite data from google earth engine using Earth Engine API.
([Documentation](https://developers.google.com/earth-engine/apidocs/ee-imagecollection))
Some ([examples](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api))
The ([GitHub ofGoogle Earth Engine Python API](https://github.com/google/earthengine-api))

This code has the following steps:
- log in [Google Earth Engine](https://earthengine.google.com/) and create or link to a project
- input parameters (dates, bands, Data Collection, AOI...)
- run a code that downloads automatically the images (.tif)

Prerequisites are:
- Have a google account and log into Google Earth Engine
- Have space for the output data

## Authentification
This part permits to authenticate into Google Earth Engine and to create or initialize a project.

Authenticate may use your default navigator to ask for permitions.

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate(auth_mode='localhost')

True

Create/link to project : 

- go on [Google earth engine/platform/](https://code.earthengine.google.com/)
- create your project
- run the following code: you can change the name of your project in the following cell.

In [2]:

# TODO change your project name
project="ee-aletschpy"


ee.Initialize(project=project)

## Parameters

This section will import the libraries and ask for your parameters.
If something is wrong with the imports, please make sure you are in an adapted virtual environment using the requirements.txt.

In [3]:
#imports
from datetime import timedelta
import geemap.core as geemap
import json
import geojson
import geopandas as gpd
import requests
import numpy as np
import io

Please add here your parameters for data download.
- Satellite_Collection: the collection you want to search into. The list and descriptions of collections can be found on the [Dataset webpage](https://developers.google.com/earth-engine/datasets?hl=fr). Please make sure that the time range you are looking for it covered by the dataset, look also for the names of the useful bands. This script is working with COPERNICUS/S2_HARMONIZED or LANDSAT/LT05/C02/T1_L2 but its right functionning cannot be assured for other datasets as variables as 'CLOUDY_PIXEL_PERCENTAGE' are maybe not defined.

- start_date: the starting date of your study (the format is ee.Date('YYYY-MM-DD'))
- end_date: the ending date of your study
- monthly_step: it is the time between two searches
- month_range: time range for which you are searching the best image.

For example: 'I want a picture of Aletsch glacier for each season (3 months) between 2020 and 2024 and I want to have the best image possible over 1 month' would translate in : start_date=ee.Date('2020-01-01'), end_date=ee.Date('2024-12-31'), monthly_step=3, month_range=1.

- polygon_path: if you are using a GeoJson, enter the path to it here, else write None
- shp_path: if you are using a .shp file, enter the path to it here, else write None 
    !! It only works if the .shp is surronded by  .shx,.qmd,.prj,.dbf,.cpg files.
- Bands: list of bands you want to download, please use the dataset documentation to have the right names
- output_directory: the path where the directory where you want to download the files
- Numpy_format: boolean to download in this format
- GeoTiff_format: booleen to download in this format

In [4]:
# Parameters
Satellite_Collection='COPERNICUS/S2_SR_HARMONIZED'#'LANDSAT/LT05/C02/T1_L2'
start_date=ee.Date('2021-07-01')
end_date=ee.Date('2022-08-01')


# Define the monthly timestep.
monthly_step = 12 # in months, step between two searches
month_range=2 # number of month over which you want to search for data at each timestep

##--------Area of Interest----------#
# Geojson
polygon_path= r'C:\Users\marga\Documents\EPFL\Cours\S2\Design_project\codes\Satellite_glacier\draft\polygon\Aletsch_1.geojson'

# Or Shp file
shp_path=None #"C:\Users\marga\Documents\EPFL\Cours\S2\Design_project\codes\Satellite_glacier\draft\polygon\Vevey1.shp"

##---------Bands------#
#Bands 
Bands=['B2','B3','B4']#['SR_B3', 'SR_B2', 'SR_B1']

#-----------Download-----#
# Directory to save the subsetted data
output_directory = r"C:\Users\marga\Documents\EPFL\Cours\S2\Design_project\Images"
Numpy_format=False
GeoTiff_format=True


## Script

Run all the cells to download the data

makes a list of months to iterate on

In [5]:
date_range= ee.DateRange(start_date, end_date) #UTC
# Calculate the number of months between the start and end dates.
months_diff = end_date.difference(start_date, 'month')

# Generate a sequence of months.
months = ee.List.sequence(0, months_diff.subtract(1), monthly_step).getInfo()
print(months)

[0, 12]


In [6]:
import numpy as np
from skimage import exposure
from skimage.io import imread, imsave
import matplotlib.pyplot as plt

def get_normalized_image(image, percentiles=(2, 98)):
    """
    Rescale image to values between 0 to 255 (capping outlier values) 
    
    Parameters
    ==================
    image: Numpy array
        Image numpy array with shape (height, width, num_bands)
    
    percentiles: tuple
        Tuple of min and max percentiles to cap outlier values
    
    Returns
    ==================
    output: Numpy array
        Normalized image numpy array
    
    """
    output = np.zeros_like(image)
    for k in range(image.shape[2]): # for each band
        p_min, p_max = np.percentile(image[:, :, k], percentiles)
        output[:, :, k] = exposure.rescale_intensity(image[:, :, k], 
                            in_range=(p_min, p_max), out_range=(0, 255))
    return output.astype(np.uint8)


Prepare the Area Of Interest from either geojson or shp file.

In [7]:
if (polygon_path==None )& (shp_path==None):
      print("!!! Specify an area of interest using .geojson or .shp file!!")
      aoi=ee.Geometry.BBox(0.1,0.1,0.1,0.1)

if polygon_path!=None:
    print("geoJson loading")
    with open(polygon_path, 'r') as file:
            geojson_data = geojson.load(file)
    coords = geojson_data['features'][0]['geometry']['coordinates']
    print(geojson_data)
    aoi=ee.Geometry.Polygon(coords)

if shp_path!=None:
      print("Shapefile loading")
      # Read the shapefile using geopandas
      gdf = gpd.read_file(shp_path)
      # Convert the geodataframe to a GeoJSON file
      geojson_data=gdf.to_json()
      # Parse GeoJSON string into a Python dictionary
      geojson_dict = json.loads(geojson_data)
      coords = geojson_dict['features'][0]['geometry']['coordinates']
      print(coords)
      aoi=ee.Geometry.Polygon(coords)

# Convert the GeoJSON to an ee.Geometry object


Map = geemap.Map()
# Add the image to the map.
Map.add_layer(aoi,None, 'Area of interest')
Map.centerObject(aoi, 11)
display(Map)


geoJson loading
{"features": [{"geometry": {"coordinates": [[[8.035762, 46.395128], [8.077107, 46.417245], [8.097815, 46.440195], [8.092364, 46.467013], [8.087227, 46.48803], [8.073553, 46.505892], [8.067174, 46.516889], [8.032211, 46.519326], [8.007453, 46.501222], [8.018197, 46.488766], [8.033307, 46.482761], [8.05474, 46.467051], [8.055246, 46.447121], [8.051066, 46.430177], [8.034506, 46.416556], [8.024673, 46.403873], [8.035762, 46.395128]]], "type": "Polygon"}, "properties": {}, "type": "Feature"}], "type": "FeatureCollection"}


Map(center=[46.465198691430835, 8.0588740665255], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

Download script: for each range of search time, search in the image collection to get an Image batch given the AOI and the dates. After that, sort the image batch by CLOUDY_PIXEL_PERCENTAGE and takes the first one. Add it as a layer to the visualization map then download it.


If you get an error and that the ImageCollection is empty, then you have to be careful about your dataset. It may not cover the time range or the names of the bands are not correct.

In [8]:
# Iterate over the sequence of months.
for month in months :
    current_start = start_date.advance(month, 'month')
    current_end = current_start.advance(month_range, 'month')
    # Print the start and end dates.
    print(f'Processing month: {current_start.format("YYYY-MM-dd").getInfo()} to {current_end.format("YYYY-MM-dd").getInfo()}')
    

    # Search image
    Image_batch=ee.ImageCollection(Satellite_Collection) .filterBounds(aoi) .filterDate(current_start, current_end)
    # Check if the collection is empty
    collection_size = Image_batch.size().getInfo()

    if collection_size == 0:
        print("The ImageCollection is empty.")
    else:
     print(f"The ImageCollection contains {collection_size} images.")

    img = ee.Image( Image_batch.sort('CLOUDY_PIXEL_PERCENTAGE',ascending=True)
                       .first() 
                       .clip(aoi)
                       .select(Bands),)
    

    #Visualisation

    vizParams = {
    'bands': Bands[:3], #Red, Green, Blue bands
    }
    Map.addLayer(img, vizParams, f'{current_start.format("YYYY-MM-dd").getInfo()} to {current_end.format("YYYY-MM-dd").getInfo()}')

    # Multi-band GeoTIFF file.
    if GeoTiff_format:
        url = img.getDownloadUrl({
            'bands': Bands,
            'region': aoi,
            'scale': 20,
            'format': 'GEO_TIFF'
        })
        response = requests.get(url)
        with open(output_directory+ f'/multi_band_{current_start.format("YYYY-MM-dd").getInfo()}.tif', 'wb') as fd:
            fd.write(response.content)
        arr = np.array(imread(output_directory+ f'/multi_band_{current_start.format("YYYY-MM-dd").getInfo()}.tif'))
        # TODO: normalize images for visualization
        arr_norm = get_normalized_image(arr)
        imsave(output_directory+ f'/multi_band_{current_start.format("YYYY-MM-dd").getInfo()}_norm.tif',arr_norm)
        
    if Numpy_format:
       url = img.getDownloadUrl({
            'bands': Bands,
            'region': aoi,
            'scale': 20,
            'format': 'NPY'})
       response = requests.get(url)
       data = np.load(io.BytesIO(response.content),allow_pickle=True)



Processing month: 2021-07-01 to 2021-09-01
The ImageCollection contains 24 images.


<tifffile.TiffPage 0 @8> parsing GDAL_NODATA tag raised ValueError("invalid literal for int() with base 10: '0.0'")


Processing month: 2022-07-01 to 2022-09-01
The ImageCollection contains 24 images.


<tifffile.TiffPage 0 @8> parsing GDAL_NODATA tag raised ValueError("invalid literal for int() with base 10: '0.0'")


Display the map with all the layers of the downloaded images. You can then choose the 3 band view (choose Stretch=100%) to view the images.

In [9]:
Map.centerObject(aoi, 11)
display(Map)

Map(center=[46.46519869143197, 8.058874066523757], controls=(ZoomControl(options=['position', 'zoom_in_text', …